In [2]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import time
from pathlib import Path
import cv2
from pprint import pprint


In [2]:
def gps2xy(lon, lat):
    # GPS coordinate: 121.0047308 30.8856365最西 121.9724096 30.8997416最东 121.2702538 30.7056483最南 121.3004672 31.5001065最北
    # raster_loc = 1766 685最南 32 742最北 1340 1995最东 1365 192最西
    
    # calculate coefficient k and b
    """
    gpsX = np.array([[103.9925022, 104.088888], [688, 1729]])
    gpsY = np.array([[30.7097699, 30.590786], [856, 2144]])

    diffX = gpsX[:, 1] - gpsX[:, 0]
    kx = diffX[1] / diffX[0]
    bx = gpsX[1, 0] - gpsX[0, 0] * kx

    diffY = gpsY[:, 1] - gpsY[:, 0]
    ky = diffY[1] / diffY[0]
    by = gpsY[1, 0] - gpsY[0, 0] * ky
    print(kx, bx) 
    print(ky, by)
    """
    kx = 1896.250418888416 
    bx = -229280.27146698005
    ky = -2215.827560963539
    by = 69829.24459460145
 
 
    coordX = np.round(kx * lon + bx).astype(int)
    coordY = np.round(ky * lat + by).astype(int)
    # print(coordX, coordY)
    return (coordX, coordY) 
 

In [3]:
def get_region_label(lon, lat):
    map_table = np.loadtxt('region_labeled.csv')
    (coordX, coordY) = gps2xy(lon, lat)
    #print(coordX, coordY)
    labels = np.zeros(lon.shape[0])
    for k in range(len(coordX)):
        try:
            label = map_table[coordX[k], coordY[k]]
            labels[k] = label
        except IndexError:
            labels[k] = 0
    return labels.astype(int)

In [4]:
def sort_time_by_id(data):
    data.sort_values('time', inplace=True) #按时间排序
    #print(data.sort_values)
    #return data.groupby('id').apply(lambda x: x.sort_values('time'))
#     dd = data.groupby('id')
#     aa = []
#     for key, values in dd:
#         values.sort_values('time',inplace=True)
#         aa.append(values)
#     return aa
    return data.groupby('id') #按id分组

In [5]:
def extract_hot_point(df, dst):     
    # df is a groupby object
    line_num = 1

    counter = 0
    for key, values in df:
        rows = values.itertuples()
        last_row = next(rows) # last_row是第一行，然后 rows从第二行开始迭代
        hot_line = []
        for row in rows:
            counter += 1
            if counter % 500000 == 0:
                print("processed {},0000 lines".format(int(counter / 10000)))
            if bool(row.empty) ^ bool(last_row.empty):
                    # 0 -> 1: leave
                    # 1 -> 0: arrive
                hot_line.append(row[0])
                line_num += 1
                last_row = row

        values = values.loc[hot_line]

        lon_array = values['lng'].as_matrix()
        lat_array = values['lat'].as_matrix()
        
        region_id = get_region_label(lon_array, lat_array)
        values['region'] = region_id

        mask = (values.region != 0) & (values.region != 1)
        values = values[mask]
        values.to_csv(dst, index=False, header=False, mode='a+') #mode='a+' 可追加写入
        #print(values.head(5))
    stat = False
    if stat:
        print("total " + str(line_num) + " lines.")


In [6]:
def process_data(src):
    processed_dir = Path(r'../Data/Temp/processed')
    filename =  src.name + '.hotpoint'
    save_dir = processed_dir / filename
    new_file(save_dir)

    data = load_gps(src)
    print("sorting time of {}".format(src))
    data_sorted = sort_time_by_id(data)
    print("sort complete, hotpoint extracting.")
    extract_hot_point(data_sorted, save_dir)

In [7]:
path = r'test.csv'
data = pd.read_csv(path,sep=',',chunksize = 10000,iterator=True)
chunks = []
i=0
for chunk in data:
    chunks.append(chunk)
    i+=1

df = pd.concat(chunks, ignore_index=True)



In [8]:
data_sorted = sort_time_by_id(df)

In [ ]:
extract_hot_point(data_sorted, Path(r'E:\Program Files\jupyter_code\code\data\total.txt'))

E:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
E:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


processed 50,0000 lines
processed 100,0000 lines
processed 150,0000 lines
processed 200,0000 lines
processed 250,0000 lines
processed 300,0000 lines
processed 350,0000 lines
processed 400,0000 lines
processed 450,0000 lines
processed 500,0000 lines
processed 550,0000 lines
processed 600,0000 lines
processed 650,0000 lines
processed 700,0000 lines
processed 750,0000 lines
processed 800,0000 lines
processed 850,0000 lines
processed 900,0000 lines
processed 950,0000 lines
processed 1000,0000 lines
processed 1050,0000 lines
processed 1100,0000 lines
processed 1150,0000 lines
processed 1200,0000 lines
processed 1250,0000 lines
processed 1300,0000 lines
processed 1350,0000 lines
processed 1400,0000 lines
processed 1450,0000 lines
processed 1500,0000 lines
processed 1550,0000 lines
processed 1600,0000 lines
processed 1650,0000 lines
processed 1700,0000 lines
processed 1750,0000 lines
processed 1800,0000 lines
processed 1850,0000 lines
processed 1900,0000 lines
processed 1950,0000 lines
proces

In [3]:
def load_hp(src):
    src = str(src)
    try:
        print("reading from " + src)
        data = pd.read_table(src, names=['id', 'status', 'time', 'latitude', 'longitude', 'region'], 
        sep=',', engine='c', float_precision='high')
        print(src + " is sucessfully loaded!")
        return data
    except:
        print(src + " is not exist.")

In [29]:
def extract_hours(arr):
    #@param arr: '2015-04-01 14:14:53'
    #print(arr.head(10))
    #print(arr)
    partitioned = arr.time.split(' ')
    if len(partitioned)<2:
        return 1
    return 0

In [31]:
data = load_hp('data/total.txt')
it = data.itertuples()
a = []
for row in it:
    #print(row[0])
    if extract_hours(row):
        a.append(row[0])
print(a)
aa = data.drop(index=a)
print('1')
aa.to_csv('data/total2.txt', index=False, header=False) 

reading from data/total.txt


E:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead.
  


data/total.txt is sucessfully loaded!
[883, 4225, 4226, 5006, 6963, 8106, 13400, 31146, 32178, 32179, 32180, 32181, 32182, 32183, 32184, 32185, 32186, 32187, 34373, 34374, 34375, 34537, 51498, 52317, 52318, 61826, 70801, 70802, 70803, 70804, 70805, 75019, 82505, 83195, 93445, 98111, 98112, 98659, 98660, 112300, 112301, 114479, 125027, 125808, 126027, 134960, 134961, 138278, 145281, 146914, 147163, 148673, 149956, 149957, 150695, 158462, 164617, 167286, 167287, 171588, 171589, 171590, 171856, 173339, 174149, 174865, 175402, 181584, 182023, 183027, 188715, 188827, 196654, 196655, 196656, 196657, 208592, 211924, 214184, 223437, 223733, 227523, 229000, 229236, 232647, 235552, 235651, 238259, 242243, 253142, 254847, 255061, 260845, 261026, 261375, 262307, 262308, 262309, 262310, 262311, 262312, 262313, 262314, 262315, 262316, 262317, 262318, 262319, 262320, 262321, 262322, 262323, 262324, 262325, 262326, 262327, 262328, 262329, 262330, 262331, 262332, 262333, 262334, 262335, 262336, 262695,

def main():
    # flag: True for preprocess the raw data, sort by time and drop duplicate GPS coordinates
    doPreprocess = True
    #flag: True for build mobility pattern, extract 

    MAX_CORE = cpu_count() - 1
    
    start_time = time.time()
    if doPreprocess:
        source_dir = Path(r'E:\Program Files\jupyter_code\出租车\data')
        #source_dir = Path(r'/home/dlbox/Documents/func_region/Data/Temp/gcj09')

        # data_range = range(24, 30+1)
        # days: 24-30
        
        source_dirs = [x for x in source_dir.glob('*.txt')]
        ## process the raw data, read, sort, and drop duplicates
        ##sort data by time groupby car ID
        p = Pool(processes=7)
        p.map(process_data, source_dirs)
        p.close()
        p.join()
    finish_time = time.time()
    print("used {}s".format(finish_time - start_time))
